# Table of Contents
* [1. imports](#1.-imports)
* [2. Viconary server data](#2.-Viconary-server-data)
	* [2.1 load data](#2.1-load-data)
		* [2.1.1 alternate ways to filter games](#2.1.1-alternate-ways-to-filter-games)
	* [2.2 analysis- prod](#2.2-analysis--prod)
	* [2.3 looking into dupes](#2.3-looking-into-dupes)
	* [2.4 viz games](#2.4-viz-games)


# 1. imports

In [1]:
%%capture
import numpy as np
import pandas as pd
import scipy.stats as st
from tqdm import tqdm
import datetime

from copy import deepcopy

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
import matplotlib.cm as cm
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light solarized
#%base16_mplrc dark solarized
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['figure.figsize'] = (16.0, 10.0)

from game_analysis.viconary import useful_cols
from game_analysis.viconary import make_standard_fig
from game_analysis.viconary import load_hit_data
from game_analysis.viconary import plot_submit_times
from game_analysis.viconary import plot_task_durations
from game_analysis.viconary import comp_task_duration
from game_analysis.viconary import comp_batch_time_stats
from game_analysis.viconary import load_games_data
from game_analysis.viconary import filter_games_by_date
from game_analysis.viconary import filter_games_by_batch_id
from game_analysis.viconary import filter_games_by_batch_prefix
from game_analysis.viconary import display_game_vid
from game_analysis.viconary import display_source_vid

# 2. Viconary server data

## 2.1 load data

In [117]:
games_dir = 'ai2-vision-viconary/games/'
full_games_df = load_games_data(games_dir)

In [118]:
# games_df_by_batch = filter_games_by_batch_prefix(full_games_df, 'vic_prod_2_200')
games_df_by_batch = filter_games_by_batch_prefix(full_games_df, 'vic_prod_4_1000')
games_df = games_df_by_batch

# full_games_df = full_games_df[~(full_games_df['drawer_assignment_id'] == '')]
# games_df = full_games_df[full_games_df['drawer_hit_batch_id'] == 'vic_prod_2_100']

In [120]:
# games_df.shape

In [72]:
submit_time = datetime.datetime(2018, 8, 23, 10, 30)

In [73]:
# games_df = filter_games_by_date(full_games_df, submit_time=submit_time, hours_pre=10, hours_post=10)[useful_cols]

In [74]:
games_df.shape

(521, 30)

In [75]:
working_df = deepcopy(games_df)

### 2.1.1 alternate ways to filter games

In [787]:
# filter_batch_id = ['third_qualification_test', 'fourth_qualification_test']# filter_b 
# filter_batch_id = ['prod_b1_300', 'prod_b2_500', 'prod_b3_500', 'prod_b4_1500', 'prod_b5_1000', 'prod_b6_2000', 'prod_b7_2000', 'prod_b8_1000', 'prod_b9_2000']
# # filter_batch_id = ['prod_b9_2000']
# games_df_by_batch = filter_games_by_batch_id(full_games_df, filter_batch_id)
# games_df = games_df_by_batch

In [838]:
# submit_time = datetime.datetime(2018, 5, 26, 0, 45)
# stop_time = datetime.datetime(2018, 5, 26, 23, 45)

# games_df_by_date = filter_games_by_date(full_games_df, submit_time , 1, 24 )
# supplement_with_hit_data(games_df_by_date, worker_lookup)

# games_df = pd.concat([games_df_by_date, games_df_by_batch])

## 2.2 analysis- prod

In [79]:
working_df.drawer_hit_batch_id.value_counts()

vic_prod_4_1000    521
Name: drawer_hit_batch_id, dtype: int64

In [80]:
assignement_returned_df = pd.read_pickle('hitdata.pkl')

In [81]:
in_game_data = set(working_df.drawer_assignment_id.tolist())

In [82]:
in_game_data_df = assignement_returned_df[assignement_returned_df.AssignmentId.isin(in_game_data)]
missing_from_game_data = assignement_returned_df[~assignement_returned_df.AssignmentId.isin(in_game_data)]

In [83]:
# missing_from_game_data

In [84]:
# working_df.id.value_counts()

In [101]:
worker_focus_df = deepcopy(working_df)

In [113]:
worker_focus_df = worker_focus_df[['drawer_worker_id', 'duration_in_min']]
worker_focus_df['count'] = np.ones_like(worker_focus_df['drawer_worker_id']) * .60
worker_avg_score = worker_focus_df.groupby('drawer_worker_id').aggregate(['sum'])

worker_avg_score['total_hours'] = worker_avg_score['duration_in_min']['sum'].apply(lambda x: x/60)
worker_avg_score['avg_hourly_rate'] = worker_avg_score['count']['sum'] / worker_avg_score['total_hours']
worker_avg_score.sort_values('avg_hourly_rate', ascending=False).head()

,duration_in_min,count,total_hours,avg_hourly_rate
,sum,sum,,
drawer_worker_id,,,,
A6U2C66WQ7QQN,3.25,1.8,0.054167,33.230769
A3TUJHF9LW3M8N,3.76,1.8,0.062667,28.723404
A25XXQPMBBDAV3,38.79,16.8,0.646500,25.986079
A1LRS8BYH64JOH,25.43,9.0,0.423833,21.234762
A3UAD8T88XUPI4,159.08,54.6,2.651333,20.593412


In [114]:
# _ = worker_avg_score.avg_hourly_rate.hist()

In [116]:
60/40

1.5

In [115]:
worker_avg_score.avg_hourly_rate.median() 

8.85968460437168

In [108]:
working_df.number_events.min()

3

In [26]:
# _ = working_df.number_events.hist()

In [58]:
game_event_dict = {}
for g in working_df.to_dict(orient='record'):
    game_event_dict[g['id']] = [e['event_name'] for e in g['events']]

no_remove_games = set([g for g, events in game_event_dict.items() if 'removeIconFromServer' not in events])
no_remove_game_df = focus_df[focus_df['id'].isin(no_remove_games)]

no_remove_game_df.shape[0] / focus_df.shape[0]

0.33695652173913043

In [65]:
all_events = pd.Series([e for g in game_event_dict.values() for e in g])

In [28]:
# all_events.value_counts()

In [ ]:
np.pad()

## 2.3 looking into dupes

In [80]:
dupe_vid_counts = focus_df.video.value_counts()
dupe_vid_counts = dupe_vid_counts[dupe_vid_counts > 1]

In [67]:
dupevid = dupe_vid_counts.index[0]

In [82]:
focus_df[working_df.video == dupevid]

,id,video,drawer,drawer_assignment_id,drawer_hit_batch_id,ended_at,events,state
13,game.91fbcd12-374b-4118-a85c-3669435394ad,WU8I6_cropped_c108_s6_5_e16_0,user.8a65726d,3R9WASFE203P3P0VEGQSYTRSQ0PFZX,batch_1,2018-07-13T17:18:39.375Z,"[{'y': 136.40625, 'x': 165.40625, 'username': ...",finished
303,game.076e428b-ac62-4a59-8c55-eadb1241b55a,WU8I6_cropped_c108_s6_5_e16_0,user.ed3b3099,3URFVVM1665B6BRZNQUPE6I7LPIUZ9,batch_1,2018-07-12T21:30:47.105Z,"[{'guessCorrect': False, 'event_name': 'endGam...",finished
387,game.95cff519-a8bd-411d-b45d-f23eefb73466,WU8I6_cropped_c108_s6_5_e16_0,user.1bb16361,3DBQWDE4Y7L3K3TN5G2PHUT6Q7JN5E,batch_1,2018-07-12T23:20:48.459Z,"[{'width': '80px', 'name': 'man', 'x': 65.7395...",finished
434,game.79625c4e-7137-4145-ac91-d73a1726acfa,WU8I6_cropped_c108_s6_5_e16_0,user.64e34c13,39U1BHVTDMEY11B3OT08BW1E1T3T3K,batch_1,2018-07-12T23:02:48.237Z,"[{'x': 210.0625, 'event_name': 'placingIconFro...",finished


In [88]:
pickled_hits =!  ls hit_data/
pickled_hits = [f for f in pickled_hits if 'prod' in f]

In [110]:
hits = []
for ph in pickled_hits:
    pf = 'hit_data/' + ph
    hits.extend(pd.read_pickle(pf))

In [113]:
video_names_hits = [h['HIT']['Question'].split('.mp4')[0].split('/')[-1] for h in hits]
video_names_hits = pd.Series(video_names_hits)

In [118]:
more_than_1 = video_names_hits.value_counts()[video_names_hits.value_counts() == 2]

In [95]:
focus_df

,id,drawer_assignment_id,drawer_worker_id,drawer_hit_batch_id,number_events,game_duration,events,duration_in_min,videoFilename,video
0,game.291e7c94-d44e-4e3e-a610-d7cd54b73cfe,34Q075JO1Y01BP0X3NYF3SKDIR001V,A2FUMA4UR6S920,batch_1,16,00:01:41.131000,"[{'height': '200px', 'y': 177.40277099609375, ...",1.69,https://s3-us-west-2.amazonaws.com/ai2-vision-...,PIMO4_cropped_c150_s1_8_e10_3
1,game.cb4f034e-2ad4-428c-849d-a18c265f4317,3KOPY89HM9P4NYFYCZM49QE6FKN3J0,A12HZGOZQD5YK7,batch_1,70,00:04:34.098000,"[{'event_name': 'placingIconFromServer', 'src'...",4.57,https://s3-us-west-2.amazonaws.com/ai2-vision-...,1M8EE_cropped_c131_c154_s22_6_e31_1
2,game.804f59dc-ad97-4081-a204-f2b50eadf522,32XVDSJFP0K53OP0BWLR681VD1R2M3,A28APJKWR06OMJ,batch_1,18,00:01:36.767000,"[{'event_name': 'placingIconFromServer', 'src'...",1.61,https://s3-us-west-2.amazonaws.com/ai2-vision-...,LY10X_cropped_c118_s3_0_e14_9
3,game.d77d6a1d-ba8a-4f59-b5b4-9189113e0fe5,39PAAFCODNNIDADM836EURTY47HVT6,A2EJLS2NDBTQHX,batch_1,61,00:11:34.399000,"[{'event_name': 'placingIconFromServer', 'src'...",11.57,https://s3-us-west-2.amazonaws.com/ai2-vision-...,QVQNK_cropped_c074_c071_c002_s10_3_e19_2
4,game.e538c676-9815-485e-aa49-067400837f6e,36ZN444YTSLXECOEDFLWLSXHXWWIO0,A25XXQPMBBDAV3,batch_1,41,00:02:28.812000,"[{'height': '200px', 'y': 170.7421875, 'event_...",2.48,https://s3-us-west-2.amazonaws.com/ai2-vision-...,ZU8Q1_cropped_c062_c112_s4_9_e16_5
5,game.4afc5eb0-fecf-4d89-8061-8d55dfa9a88c,3RXCAC0YISCGXTV3GAWX3UU13EU8GP,A2FUMA4UR6S920,batch_1,24,00:01:36.927000,"[{'height': '80px', 'y': 133.40277099609375, '...",1.62,https://s3-us-west-2.amazonaws.com/ai2-vision-...,Q0BNP_cropped_c106_s23_8_e32_0
6,game.603b5ce9-cda6-4294-8f9c-fe7a079f7977,3OB0CAO74IC9CVHWDKC9N7U4BT9YHR,A2FUMA4UR6S920,batch_1,27,00:02:42.122000,"[{'height': '200px', 'y': 102.40277099609375, ...",2.70,https://s3-us-west-2.amazonaws.com/ai2-vision-...,TTZVW_cropped_c090_s0_0_e9_5
7,game.053d8940-9be0-492e-9864-a37ef0b8cc7a,3LWJHTCVCD9GPXZ4RK7F70FPW5GFQ8,A2EJLS2NDBTQHX,batch_1,38,00:03:15.368000,"[{'height': '200px', 'y': -58.59375, 'event_na...",3.26,https://s3-us-west-2.amazonaws.com/ai2-vision-...,BBQIS_cropped_c151_s20_8_e29_1
8,game.41fdd8f9-f3dd-4ae5-a373-9f37e4f94cd2,3R0T90IZ1TZI9WLGXMC25GQKT5FGC3,A3MHYBS6PHJ5QG,batch_1,29,00:02:22.840000,"[{'height': '200px', 'y': 13.2889404296875, 'e...",2.38,https://s3-us-west-2.amazonaws.com/ai2-vision-...,KNXCB_cropped_c129_s0_0_e9_1
9,game.0a64e8cc-bca6-40bf-befa-0a0bd0a5b0dc,36PW28KO40JWW3SR7DQNZIFKGMRAEQ,ABI1EI8UQK6LK,batch_1,48,00:05:52.228000,"[{'event_name': 'placingIconFromServer', 'src'...",5.87,https://s3-us-west-2.amazonaws.com/ai2-vision-...,UJO36_cropped_c107_c119_s8_2_e16_5


In [124]:
submitted_twice = set(more_than_1.index)

In [128]:
games_subbed_twice = focus_df[focus_df['video'].isin(submitted_twice)]

In [134]:
sum(games_subbed_twice.video.value_counts() > 1)

101

In [42]:
g1 = working_df[working_df['id'].str.contains('26755')]
g2 = working_df[working_df['id'].str.contains('3523cb')]

In [49]:
vids_to_submit = pd.read_pickle('submitted_vids.pkl')

In [50]:
intended_to_submit = set(vids_to_submit.tolist())

In [46]:
act_done = set(working_df['videoFilename'].tolist())

In [33]:
# intended_to_submit.difference(act_done)

In [34]:
all_submitted_aids = pd.read_pickle('submitted_aids.pkl')

In [143]:
game_data_aids = set(focus_df['drawer_assignment_id'].tolist())

In [144]:
hit_aids = set(all_submitted_aids.tolist())

In [97]:
# hit_aids.difference(game_data_aids)

In [146]:
game_data_aids.difference(hit_aids)

{'38JBBYETQPXHUECAYCGWMFTJFXD4EM'}

In [ ]:
dupe_aids = ['38JBBYETQPXHUECAYCGWMFTJFXD4EM', '3P1L2B7AD2CZ4DWKYII9D8P85M1LOP']

In [118]:
g1['ended_at']

134   2018-07-18 10:53:38
Name: ended_at, dtype: datetime64[ns]

In [119]:
g2['ended_at']

60   2018-07-18 12:23:03.531
Name: ended_at, dtype: datetime64[ns]

In [37]:
# display_game_vid(g1['id'].iloc[0])

In [19]:
focus_df['drawer_worker_id'].to_pickle('viconary_first_test_wids.pkl')

In [47]:
focus_df['duration_in_min'].median()

4.25

In [49]:
# focus_df['number_events'].mean()

53.25

In [38]:
# fig_labels = {
#     'fig_title': 'Time Spent on Task',
#       'x_label': 'task duration (min)',
#       'y_label': '# tasks'
# }
# make_standard_fig(focus_df['duration_in_min'].hist(), fig_labels)

## 2.4 viz games

In [48]:
examine_id = 'game.5f11caa3-082c-4b59-9725-2e4abb970b4a'
examine_game = focus_df[focus_df['id'] == examine_id].iloc[0]

In [54]:
min_cols = ['id', 'duration_in_min', 'number_events']
# examine_game = working_df[working_df['id'] == examine_id].iloc[0]
examine_game = working_df[min_cols].sample(1).iloc[0]

pd.DataFrame(examine_game)

,62
id,game.19dca149-d174-43dd-a7ec-2a22978e307f
duration_in_min,3.85
number_events,48


In [56]:
# display_source_vid(test_vid.video)